In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
list_file = []
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        list_file.append(os.path.join(dirname, filename))

/kaggle/input/training-data-cfm/X_test_m4HAPAP.csv
/kaggle/input/training-data-cfm/y_train_or6m3Ta.csv
/kaggle/input/training-data-cfm/X_train_N1UvY30.csv
/kaggle/input/best_mod/pytorch/default/1/best_mod_cfm.pth
/kaggle/input/small-dataset/small_X_test.csv
/kaggle/input/small-dataset/small_X_train.csv
/kaggle/input/small-dataset/small_y_train.csv
/kaggle/input/execfiles/exec.py
/kaggle/input/execfiles/train.py
/kaggle/input/execfiles/model.py
/kaggle/input/execfiles/load.py
/kaggle/input/execfiles/report_end.py
/kaggle/input/execfiles/__init__.py
/kaggle/input/execfiles/loader.py
/kaggle/input/optimization-cfm/small_X_test.csv
/kaggle/input/optimization-cfm/small_X_train.csv
/kaggle/input/optimization-cfm/small_y_train.csv


**Loader**

In [6]:
import sys
sys.path.append("/kaggle/input/execfiles")

import torch
from model import AdvancedRNN, gru_lstm
from train import train
from loader import get_loaders
from load import load_data
from report_end import  make_predictions
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
import os 



# Load Data

In [127]:
dict_feature_eng = dict(use_microstructure=True,
                use_imbalance=True,
                use_price_dynamics=True,
                use_momentum=True,
                momentum_windows=[5, 20],
                use_flux=True,
                use_directional=True,
                use_time=True,
                use_log=True,
                corr_threshold=None)

dict_load_data = dict(dummy = False, normalize=True, filter=True, print_shape= True, threshold = 10, dict_feature_eng = dict_feature_eng)


loaded_data = load_data(**dict_load_data)
X_train, y, X_test = loaded_data

KeyboardInterrupt: 

# Loader

In [98]:
dict_loaders = dict(batch_size=32, test_size=0.1, seed=42, shuffle = True)

train_loader,val_loader,test_loader = get_loaders(loaded_data = loaded_data, **dict_loaders)

# Model

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_params = dict(num_features= 16,
                num_classes= 24,
                embed_features= [0, 2, 3, 4],
                num_embed_features= [6, 3, 2, 2],
                encode_features= [1],
                embedding_dim= 8,
                hidden_dim= 256,  # renommé d_hidden -> hidden_dim
                dropout= 0.15,
                rnn_type= "LSTM",
                attention= True,
                use_transformer= True,
                nhead= 4,
                num_transformer_layers= 2,
                device= device)

model = AdvancedRNN(**model_params)
if "/kaggle/input/best_mod/pytorch/default/1/best_mod_cfm.pth" in list_file:
    state_dict = torch.load("/kaggle/input/best_mod/pytorch/default/1/best_mod_cfm.pth", map_location='cuda')  # ou 'cpu'
    model.load_state_dict(state_dict)
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))



AdvancedRNN(
  (embeddings): ModuleList(
    (0): Embedding(6, 8)
    (1): Embedding(3, 8)
    (2-3): 2 x Embedding(2, 8)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=44, out_features=44, bias=True)
        )
        (linear1): Linear(in_features=44, out_features=88, bias=True)
        (dropout): Dropout(p=0.15, inplace=False)
        (linear2): Linear(in_features=88, out_features=44, bias=True)
        (norm1): LayerNorm((44,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((44,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.15, inplace=False)
        (dropout2): Dropout(p=0.15, inplace=False)
      )
    )
  )
  (rnn): LSTM(44, 256, batch_first=True, bidirectional=True)
  (attention_weights): Linear(in_features=512, out_features=1, bias=True)
  (fc): Sequential(
    (0)

# Training

In [ ]:
training_params =  dict(
                    lr= 6e-5,
                    weight_decay= 3e-4,
                    gamma= 0.6,
                    step_size= 5,
                    num_epochs= 100,
                    early_stop_n= 10,
                    patience= 10,
                    device= device, 
                    scheduler= {
                        "mode": "min",
                        "factor": 0.6,
                        "patience": 3,
                        "verbose": True
                    })

# === Optimiser === #
optimizer = Adam(model.parameters(), lr=training_params["lr"], weight_decay=training_params["weight_decay"])

# === Scheduler === #
scheduler = ReduceLROnPlateau(optimizer, **training_params["scheduler"])

# === Training === #
train(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    scheduler=scheduler,
    num_epochs=training_params["num_epochs"],
    device=training_params["device"],
    patience=training_params["patience"],
    n=training_params["early_stop_n"]
)
# Save Model
MODEL_OUT = '/kaggle/working/'
torch.save(model.state_dict(), os.path.join(MODEL_OUT,'best_mod_cfm_after_feng.pth'))

# Make Predictions

In [12]:
Y = make_predictions(model,test_loader)
print(Y)

     eqt_code_cat
0               7
1               3
2              19
3              19
4              22
..            ...
811            22
812            15
813            10
814            17
815             7

[816 rows x 1 columns]


In [111]:
import torch


# === Hyperparamètres === #
num_features = 16
num_classes = 24
embed_features = [0, 2, 3, 4]
num_embed_features = [6, 3, 2, 2]
encode_features = [1]

embedding_dim = 8
d_hidden = 256
num_layers = 1
dropout = 0.15
rnn_type = 'LSTM'
attention = True
use_transformer = True
nhead = 4
num_transformer_layers = 2


# === Détection du device === #
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Initialisation du modèle === #
model = AdvancedRNN(
    num_features=num_features,
    num_classes=num_classes,              # renommé
    embed_features=embed_features,
    num_embed_features=num_embed_features,
    encode_features=encode_features,
    embedding_dim=embedding_dim,
    hidden_dim=d_hidden,                  # renommé
    dropout=dropout,
    rnn_type=rnn_type,
    attention=attention,
    use_transformer=use_transformer,
    nhead=nhead,
    num_transformer_layers=num_transformer_layers,
    device=device
)
if "/kaggle/input/best_mod/pytorch/default/1/best_mod_cfm.pth" in list_file:
    state_dict = torch.load("/kaggle/input/best_mod/pytorch/default/1/best_mod_cfm.pth", map_location='cuda')  # ou 'cpu'
    model.load_state_dict(state_dict)



In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
# Optimisation

lr = 1e-4
weight_decay = 3e-4
gamma = 0.6
step_size = 5
num_epochs = 10
early_stop_n = 10  # patience pour l'early stopping
patience = 10


# === Optimiseur et scheduler === #
optimizer = Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
# scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
scheduler = ReduceLROnPlateau(optimizer,mode='min',factor= gamma,patience=3)

# === Entraînement === #
train(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    scheduler=scheduler,
    num_epochs=num_epochs,
    device=device,
    patience = patience, 
    n=early_stop_n  # nombre d'epochs sans amélioration avant arrêt
)
MODEL_OUT = '/kaggle/working/'
torch.save(model.state_dict(), os.path.join(MODEL_OUT,'best_mod_cfm.pth'))

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


🔹 Training on device: cuda


📌 Epoch 2: Train Loss: 0.4476, Train Acc: 0.8415
✅ Validation: Loss: 0.3816, Accuracy: 0.8694
⚠️ Early stopping patience: 1/10


Epoch 3/10:  82%|████████▏ | 3612/4427 [01:07<00:15, 53.45it/s, Train Loss=1595.5532, Train Acc=0.6886]

In [ ]:
print (model)

In [121]:
def make_predictions(model, X_test, device="cuda" if torch.cuda.is_available() else "cpu"):
    all_predictions = []  
    all_probabilities = [] 

    model.to(device)
    model.eval()
    with torch.no_grad():
        for batch in X_test:
            # Vérifie le type du batch
            if isinstance(batch, (list, tuple)):
                batch_inputs = batch[0]  # Prend le premier élément
            else:
                batch_inputs = batch

            batch_inputs = batch_inputs.to(device, dtype=torch.float32)  # OK maintenant

            outputs = model(batch_inputs)
            probabilities = torch.softmax(outputs, dim=1)
            predicted_classes = torch.argmax(probabilities, dim=1)

            all_predictions.append(predicted_classes.cpu().numpy())
            all_probabilities.append(probabilities.cpu().numpy())

    all_predictions = np.concatenate(all_predictions)
    all_probabilities = np.concatenate(all_probabilities)

    df_prediction = pd.DataFrame({"eqt_code_cat": all_predictions})
    df_prob = pd.DataFrame(all_probabilities, columns=[f"class_{i}_prob" for i in range(all_probabilities.shape[1])])
    df_prob["predicted_class"] = all_predictions
    df_prob["max_prob"] = df_prob[[f"class_{i}_prob" for i in range(all_probabilities.shape[1])]].max(axis=1)

    sorted_probs = np.sort(all_probabilities, axis=1)[:, ::-1]
    df_prob["second_max_prob"] = sorted_probs[:, 1]
    df_prob["max_minus_second"] = df_prob["max_prob"] - df_prob["second_max_prob"]
    df_prob["entropy"] = entropy(all_probabilities, axis=1)

    df_prediction.to_csv("/kaggle/working/y_predict.csv", index=False)
    df_prob.to_csv("/kaggle/working/proba_details.csv", index=False)

    return df_prediction, df_prob


In [65]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

def prepare_pseudo_labels(df_prob, X_unlabeled, model, threshold=0.9):
    """
    Génère un dataset pseudo-étiqueté à partir des prédictions d'un modèle.
    
    df_prob : DataFrame avec colonnes de probas par classe et indicateurs
    X_unlabeled : données d'entrée non étiquetées (Tensor ou DataFrame)
    model : modèle entraîné
    threshold : seuil de confiance pour accepter un pseudo-label
    
    Retour :
    df_pseudo : DataFrame prêt pour l'ajout à l'entraînement
    """
    
    # Filtrer sur la confiance
    confident_mask = df_prob['max_prob'] >= threshold
    confident_indices = df_prob[confident_mask].index

    print(f"Nombre d'échantillons ajoutés : {len(confident_indices)} sur {len(df_prob)}")

    # Extraire les pseudo-labels et indicateurs
    pseudo_labels = df_prob.loc[confident_indices, 'predicted_class'].values
    confidences = df_prob.loc[confident_indices, 'max_prob'].values
    entropies = df_prob.loc[confident_indices, 'entropy'].values
    second_probs = df_prob.loc[confident_indices, 'second_max_prob'].values
    margins = df_prob.loc[confident_indices, 'max_minus_second'].values

    # Construire DataFrame pseudo-label
    df_pseudo = pd.DataFrame({
        'pseudo_label': pseudo_labels,
        'max_prob': confidences,
        'entropy': entropies,
        'second_max_prob': second_probs,
        'max_minus_second': margins
    })

    # Ajouter les features correspondantes
    X_pseudo_labeled = X_unlabeled[confident_indices]
    return X_pseudo_labeled, df_pseudo


In [96]:
X_pseudo_labeled, df_pseudo = prepare_pseudo_labels(df_prob, X_test, model, threshold=0.9)

Nombre d'échantillons ajoutés : 256 sur 816


In [122]:
df_prediction, df_prob = make_predictions(model,test_loader)
df_prob

,class_0_prob,class_1_prob,class_2_prob,class_3_prob,class_4_prob,class_5_prob,class_6_prob,class_7_prob,class_8_prob,class_9_prob,...,class_19_prob,class_20_prob,class_21_prob,class_22_prob,class_23_prob,predicted_class,max_prob,second_max_prob,max_minus_second,entropy
0,2.070337e-04,7.738247e-04,4.830919e-04,2.585734e-05,1.308414e-04,4.807984e-04,2.322775e-04,4.292259e-01,4.899747e-04,6.212350e-04,...,1.694874e-05,6.464120e-06,4.876687e-05,5.520727e-07,7.643536e-02,7,0.429226,0.223948,0.205277,1.456735
1,4.412778e-08,2.255851e-05,2.160316e-07,9.951594e-01,2.492029e-06,2.653640e-03,8.523249e-07,2.406180e-07,4.678523e-05,3.255874e-06,...,4.285978e-08,6.418047e-05,1.590437e-04,1.356402e-05,4.758763e-08,3,0.995159,0.002654,0.992506,0.036676
2,1.054914e-06,1.066540e-02,9.119313e-05,5.757163e-09,1.592373e-07,8.509778e-07,5.466239e-02,5.843077e-04,1.036412e-05,2.621825e-07,...,9.036809e-01,2.233715e-08,1.364520e-07,2.711414e-07,1.563360e-02,19,0.903681,0.054662,0.849018,0.443194
3,1.594730e-08,7.332956e-03,1.648885e-10,1.001064e-12,4.009720e-10,4.842547e-12,9.804125e-01,2.295031e-08,1.823334e-08,6.589534e-10,...,1.115199e-03,1.495563e-12,3.057469e-11,9.666078e-11,1.500369e-08,6,0.980412,0.011077,0.969336,0.113541
4,5.968463e-05,1.883935e-09,4.276681e-09,6.677072e-07,1.206044e-02,3.110668e-08,5.311274e-08,5.571531e-08,4.567823e-08,1.168293e-06,...,3.251912e-07,1.191801e-05,3.533346e-04,9.804578e-01,1.835905e-08,22,0.980458,0.012060,0.968397,0.116063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81595,7.779440e-01,1.891168e-07,7.907412e-07,6.687896e-05,1.519985e-02,9.265303e-07,8.264483e-07,2.290279e-05,3.726136e-05,6.986754e-02,...,2.189299e-06,8.851662e-04,9.598112e-02,7.773886e-03,2.028980e-07,0,0.777944,0.095981,0.681963,0.848491
81596,7.573255e-01,3.507045e-07,3.264645e-07,5.245494e-07,1.669574e-05,5.588258e-09,2.074131e-06,2.268885e-06,5.215336e-07,1.874029e-02,...,2.742100e-07,2.675837e-04,1.786072e-01,2.553474e-02,5.412899e-08,0,0.757325,0.178607,0.578718,0.766592
81597,1.107603e-05,8.617907e-01,2.817413e-05,9.838719e-07,1.893947e-06,1.248610e-06,1.018275e-01,2.552255e-04,2.886380e-04,9.956712e-06,...,3.449767e-04,3.791425e-07,6.074677e-07,4.927076e-07,1.635285e-04,1,0.861791,0.101828,0.759963,0.527349
81598,1.672865e-06,2.373340e-08,3.677909e-07,2.178252e-05,1.331505e-03,2.445391e-04,5.039130e-09,9.885269e-10,4.288259e-06,1.015929e-07,...,2.387266e-07,6.460229e-02,8.122064e-02,5.500914e-05,1.467882e-07,10,0.850694,0.081221,0.769474,0.543030


In [124]:
df_prediction.to_csv("test.csv")

In [107]:
df_prediction[df_prob['max_prob'] < x_high_1] = 1

In [109]:
x_high_1

0.6

In [108]:
df_prediction.to_csv("test_rapid60.csv")